In [1]:
import pandas as pd

from config_paths import PATHS, AUX_PATHS
from data_utils import import_data, preprocessing
from data_prep import split_data, split_idx
from evaluation import create_ground_truth, fix_dates, convert_dates
from iforest import detect_anomalies

In [2]:
ground_truth = create_ground_truth(
    AUX_PATHS, cols_to_keep=["ID", "IDpix", "Finicio", "Ffinal"]
)
ground_truth = ground_truth.pipe(fix_dates).pipe(convert_dates)
ground_truth

,ID,IDpix,Finicio,Ffinal,NAME
0,0,1,2004-07-03,2022-10-21,ESTABLE
1,0,2,2004-07-03,2022-10-21,ESTABLE
2,0,3,2004-07-03,2022-10-21,ESTABLE
3,0,4,2004-07-03,2022-10-21,ESTABLE
4,0,5,2004-07-03,2022-10-21,ESTABLE
...,...,...,...,...,...
27092,369,2437,2006-02-27,2018-03-22,VARIOS
27093,369,2438,2006-02-27,2018-03-22,VARIOS
27094,369,2439,2006-02-27,2018-03-22,VARIOS
27095,369,2440,2006-02-27,2018-03-22,VARIOS


In [3]:
df_dict = {}
for name in PATHS.keys():
    df_dict[name] = import_data(name, paths=PATHS, aux_paths=AUX_PATHS).pipe(
        preprocessing, name=name, filter_id=4
    )
    print("------------------------------------------------------")

train_df_dict, test_df_dict = split_data(df_dict, paths=PATHS, date="2016-01-01")
train_dates, test_dates = split_idx(df_dict, date="2016-01-01")

Importing 1895 timestamps and 2309 Time Series for ESTABLE.
Preprocessed data for ESTABLE. 2309 TimeSeries and 1195 TimeStamps.
------------------------------------------------------
Importing 2131 timestamps and 9270 Time Series for INCENDIO.
Preprocessed data for INCENDIO. 9270 TimeSeries and 1195 TimeStamps.
------------------------------------------------------
Importing 2370 timestamps and 6666 Time Series for SEQUIA.
Preprocessed data for SEQUIA. 6666 TimeSeries and 1195 TimeStamps.
------------------------------------------------------
Importing 1801 timestamps and 6411 Time Series for TALA.
Preprocessed data for TALA. 6411 TimeSeries and 1195 TimeStamps.
------------------------------------------------------
Importing 1689 timestamps and 2441 Time Series for VARIOS.
Preprocessed data for VARIOS. 2441 TimeSeries and 1195 TimeStamps.
------------------------------------------------------


In [4]:
anomalies_df = detect_anomalies(
    train_df_dict, test_df_dict, paths=PATHS, test_dates=test_dates
)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [5]:
(
    anomalies_df.merge(ground_truth, how="left", on=["NAME", "IDpix", "ID"]).assign(
        is_right=lambda x: (x.Finicio <= x.target_dates) & (x.target_dates <= x.Ffinal),
        P_id=lambda x: x.NAME + "-" + x.ID.astype(str),
    )
)
anomalies_df
anomalies_df.to_parquet("../data/predictions/Iforest_preds.parquet")